In [2]:
import sys
print(f'Python version:', sys.version)

Python version: 3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)]


In [39]:
# Packages
import termcolor as tc
import Bio; print('BioPython version:', Bio.__version__)

# Python packages
import os
import sys
import gc
import string
import pickle
import collections
import random

# Data Science and Computation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy # statistics
import sklearn # machine learning

# Natural language processing
import nltk

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# Local Imports
import definitions as defs

BioPython version: 1.76


In [16]:
sars_file = os.path.join(defs.DATA_DIR, 'sars_sequences.fasta')

In [17]:
sars_file

'D:\\code\\py\\corona-virus-analysis\\data\\sars_sequences.fasta'

# Helper Functions

In [41]:
def examine_distribution(sequence, title=''):
    # Calculate statistics
    mean = np.mean(sequence)
    median = np.median(sequence)
    mode = int(scipy.stats.mode(sequence).mode)
    statistics = {'Mean': mean, 'Median': median, 'Mode': mode}

    # Plot distribution
    fig, ax = plt.subplots()
    sns.distplot(sequence)
    ax.axvline(mean, color='r', linestyle='--')
    ax.axvline(median, color='g', linestyle='-')
    ax.axvline(mode, color='b', linestyle='-')

    plt.title(title.capitalize())
    plt.legend(statistics)
    plt.show()

    for key, value in statistics.items():
        print(f'\t{key}={value:.2f}')


In [7]:
handle = Entrez.efetch(db='nucleotide', id='1850859292', rettype='fasta', retmode='text')
record_b = SeqIO.read(handle, 'fasta')

In [31]:
from Bio import SeqIO
from Bio import Entrez

Entrez.email = 'filip.markoski45@gmail.com'
handle = Entrez.esearch(db='nucleotide', term='SARS-CoV-2')
search = Entrez.read(handle)

for id in search['IdList']:
    handle = Entrez.efetch(db='nucleotide', id=id, rettype='fasta', retmode='text')
    record = SeqIO.read(handle, 'fasta')
    print(record.description)
    print(len(record.seq))

2

# Code

In [37]:
?? Entrez.efetch

In [249]:
def construct_features(seq):
    features = []
    
    features.extend(construct_sequtils_features(seq, window=500))
    features.extend(construct_nucleotide_counts(seq, display=False))
    features.extend(construct_amino_acid_features(seq, display=False))
    features.extend(construct_n_gram_features(seq, window_size=3))
    features.extend(construct_fourier_wavelet_feature(seq))
    
    return np.array(features)

In [264]:
features = []

with open(sars_file) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    nucleotides = []
    descriptions = []
    rna = []
    amino_acid = []
    count = 0
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        if count == 5:
            break
        count+=1
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        nucleotides.append(seq_record.seq)
        descriptions.append(seq_record.description)
        rna_temp = seq_record.seq.transcribe()
        rna.append(rna_temp)
        amino_acid.append(rna_temp.translate())
        
        features.append(construct_features(seq_record.seq))
        
s1 = pd.Series(identifiers, name='ID')
s2 = pd.Series(lengths, name='length')
#s3 = pd.Series(nucleotides, name='Seq')
s4 = pd.Series(descriptions, name='Description')
#Gathering Series into a pandas DataFrame and rename index as ID column
sars_nucelotide = pd.DataFrame(dict(ID=s1, length=s2, Seq=nucleotides,Description=s4, RNA=rna, Amino_Acid=amino_acid)).set_index(['ID'])
sars_nucelotide

ValueError: 'M' is not a valid unambiguous letter for RNA

In [257]:
sars_nucelotide.T.loc['Seq']

ID
AY463060.1    (T, C, T, G, T, G, T, A, G, C, T, G, T, C, G, ...
KJ473811.1    (G, G, T, T, T, C, G, T, C, C, G, G, G, T, G, ...
FJ588686.1    (C, C, A, G, G, A, A, A, A, G, C, C, A, A, C, ...
KJ473816.1    (G, T, T, T, C, G, T, C, C, G, G, G, T, G, T, ...
KJ473815.1    (G, T, C, C, G, G, G, T, G, T, G, A, C, C, G, ...
Name: Seq, dtype: object

# Features 

In [261]:
from Bio.SeqUtils import *

def construct_sequtils_features(seq, window=500, display=False) -> list:
    seq = seq.transcribe()
    
    gc_content = GC(seq)
    mol_weight = molecular_weight(seq, seq_type='RNA')

    gc_skew_array = GC_skew(seq, window=window)
    
    if display:
        print(len(gc_skew_array))

    max_gc_skew = np.max(gc_skew_array)
    min_gc_skew = np.min(gc_skew_array)
    mean = np.mean(gc_skew_array)
    median = np.median(gc_skew_array)
    mode = scipy.stats.mode(gc_skew_array).mode[0]

    vector = [gc_content, mol_weight, max_gc_skew, min_gc_skew, mean, median, mode]
    # vector.extend(gc_skew_array[:60])
    return vector

len(construct_sequtils_features(seq=record.seq))

7

In [241]:
#count feature
def construct_nucleotide_counts(seq, display=False) -> list:
    seq = seq.transcribe()
    
    mapping = {
        'A': 0,
        'U': 0,
        'G': 0,
        'C': 0,
    }
    template = collections.Counter(mapping)
    counter = collections.Counter(seq)
    template =  template + counter
    
    if display:
        print(list(template.items()))
    
    vector = list(template.values())
    return vector

construct_nucleotide_counts(record.seq, display=True)

[('A', 8954), ('U', 9603), ('G', 5860), ('C', 5486)]


[8954, 9603, 5860, 5486]

In [240]:
#amino-acid distribution

def construct_amino_acid_features(seq, display=False) -> list:
    seq_AminoAcids = seq_rna.translate()
    amino_acids_dict = {'Y': 0, 'A': 0, 'Q': 0, 'D': 0, 'C': 0, 'G': 0, 'V': 0, 'T': 0, 'E': 0, 'N': 0, 
                       'K': 0, 'R': 0, 'S': 0, 'I': 0, 'H': 0, 'M': 0, 'F': 0, 'L': 0, 'W': 0, 'P': 0}
    
    # Percentages
    
    for amino_acid in amino_acids_dict:
        amino_acids_dict[amino_acid] = seq_AminoAcids.count(amino_acid)/len(seq_AminoAcids)*100
    
    vector = list(amino_acids_dict.values())
    
    if display:
        print(amino_acids_dict.values())
    
    # Counts 
    
    for amino_acid in amino_acids_dict:
        amino_acids_dict[amino_acid] = seq_AminoAcids.count(amino_acid)
    
    vector.extend(list(amino_acids_dict.values()))
    return vector
        
len(construct_amino_acid_features(record.seq, display=False))

40

In [238]:
import itertools

def construct_n_gram_features(seq, window_size=3) -> list:
    permutations = itertools.product('ACTG', repeat=window_size)
    mapping = dict.fromkeys(list(permutations), 0)

    template = collections.Counter(mapping)

    n_gram_generator = ngrams(record.seq, window_size)
    counter = collections.Counter(n_gram_generator)

    template = template + counter
    
    # getting the feature name
    # ''.join(list(counter.keys())[0])
    
    vector = list(template.values())
    return vector
    
print(len(construct_n_gram_features(record.seq, window_size=3)))

64


In [61]:
''.join(list(counter.keys())[0])

'ATT'

In [236]:
import pywt
import math
#https://pywavelets.readthedocs.io/en/0.2.2/ref/dwt-discrete-wavelet-transform.html
#mapping functon
def toi(n):
    if n=='A':
        return 0
    if n=='C':
        return 1
    if n=='G':
        return 2
    if n=='U':
        return 3
    
# determine the decomposition level
# N denotes the length of the DNA sequence
# M is the fixed length of the feature vector, and L is the decomposition level.
def decomposition_lvv(N,M):
    r = N/M
    ll = math.log(r,2)
    L = math.ceil(ll)
    return L    

# DWT reduces the dimension of CODE. WFV uses the simplest Haar wavelet to create the feature vector of DNA sequence S
def construct_fourier_wavelet_feature(seq):
    # convert to RNA sequence
    seq = seq.transcribe()
 
    code_list = list(map(toi, seq)) 
    code = ''.join(map(str, code_list))

    if len(code) < 30133:
        number = 30133 - len(code)
        s = ''.zfill(number)
        code += s
  
        
    N = len(code)
    l = decomposition_lvv(N, 2)
    # print(l)
    coeffs = pywt.wavedec(code_list, 'haar', level = l)
    return list(coeffs)

a = record.seq
b = x
result1 = construct_fourier_wavelet_feature(a)
result2 = construct_fourier_wavelet_feature(b)
print(len(result1))
print(len(result2))
for i in range(0,len(result1)):
    if len(result1[i]) == len(result2[i]):
        print(i, 'True')

15
15
0 True
1 True
2 True
3 True
4 True


In [212]:
from Bio.Seq import Seq
Seq('AAC')

Seq('AAC')

In [230]:
s= ''
s.zfill(4)

'0000'